In [ ]:
import pandas as pd
import geopandas as gpd

pd.options.plotting.backend = "plotly"
import numpy as np
import plotly.graph_objects as go


from ares_py.class_project import Project
from ares_py.tools.get_ld import get_ld

In [ ]:
from ares_py.tools.get_ld import get_ld
import pandas as pd
import numpy as np
from pathlib import Path


def kysuce_cor_preprocess():
    new = [0, np.nan, 6, 9, 12, 13, 14, 15, 18, 21, np.nan, 30]
    replace = pd.Series(data=new, index=list(range(12)), name="vals")

    ld = get_ld("projects/kysuce_korozny/00_raw/")

    for fp in ld["fp"]:
        fn = Path(fp).stem
        line = int(fn)
        fp_out = f"projects/kysuce_korozny/01_2dm/{str(line).zfill(3)}.2dm"

        with open(fp) as file:
            lines = file.readlines()

        lines = lines[:18]

        with open(fp_out, "w") as file:
            file.writelines(lines)

        df = pd.read_csv(fp, skiprows=18, header=None, sep="\t")
        for col in [0, 1, 2, 3]:
            tmp = pd.merge(df, replace, "left", left_on=col, right_index=True)
            tmp = tmp["vals"].values
            df[col] = tmp

        df = df.dropna(subset=[0, 1, 2, 3]).reset_index(drop=True)
        df = df.astype({0: int, 1: int, 2: int, 3: int})
        df.to_csv(fp_out, sep="\t", index=False, header=None, mode="a")

In [ ]:
project_name = "kysuce_korozny"

grid_skip = []
crs = 8353
res_range = [1, 10000]


atlas_input = dict(
    res_min=[0] * 4,
    res_max=[500] * 4,
    page_heights=[420, 594, 420, 594],
    page_heights_int=[297] * 4,
    map_scales=[250] * 4,
    lvl_frequency=[1, 3, 1, 3],
    z_gap=[10, 10, 10, 5],
)
prj = Project(project_name, crs=crs, res_range=res_range)

In [59]:
from ares_py.class_raw import Raw

kysuce_cor_preprocess()

prj = prj.Process_raw()

print("\n")
prj = prj.Process_apy()
prj = prj.Merge_apy_coordinates()
print("\n")

prj = prj.Gpkg_measured()

<bound method PlotAccessor.scatter of <pandas.plotting._core.PlotAccessor object at 0x000001F3687028B0>>